# Supervised Learning Challenge

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
# from pathlib import Path
pd.set_option("display.max_columns", None)

In [3]:
df1 = pd.read_csv("LoanStats_2019Q1.csv")
df2 = pd.read_csv("LoanStats_2019Q2.csv")
df3 = pd.read_csv("LoanStats_2019Q3.csv")
df4 = pd.read_csv("LoanStats_2019Q4.csv")

# Concatenate CSVs

In [4]:
train_df = pd.concat([df1, df2, df3, df4], ignore_index = True)
test_df = pd.read_csv("LoanStats_2020Q1.csv")

In [5]:
train_df = train_df.dropna(axis = "columns", how = "all")
test_df = test_df.dropna(axis = "columns", how = "all")

train_df = train_df.dropna(how = "all")
test_df = test_df.dropna(how = "all")

In [6]:
train_df.drop("emp_title", axis = 1, inplace = True)
test_df.drop("emp_title", axis = 1, inplace = True)

In [7]:
# View data
train_df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,C,C5,6 years,RENT,47000.0,Source Verified,Mar-19,Issued,n,debt_consolidation,Debt consolidation,958xx,CA,14.02,0.0,Sep-06,1.0,50.0,NaN,15.0,0.0,10687.0,19.70%,53.0,w,20000.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Apr-19,Apr-19,0.0,50.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,31546.0,3.0,2.0,1.0,2.0,10.0,20859.0,97.0,4.0,9.0,5909.0,42.0,54300.0,6.0,1.0,3.0,11.0,2103.0,23647.0,30.0,0.0,0.0,150.0,100.0,1.0,1.0,0.0,5.0,NaN,3.0,50.0,3.0,3.0,4.0,8.0,19.0,19.0,13.0,33.0,4.0,15.0,0.0,0.0,0.0,5.0,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,21225.0,21225.0,21225.0,60 months,14.74%,502.05,C,C2,10+ years,MORTGAGE,225000.0,Not Verified,Mar-19,Issued,n,credit_card,Credit card refinancing,956xx,CA,16.80,0.0,Sep-94,2.0,NaN,110.0,14.0,1.0,95868.0,87.70%,22.0,w,21225.0,21225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Apr-19,Apr-19,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,694793.0,1.0,5.0,2.0,4.0,3.0,113558.0,76.0,0.0,1.0,17353.0,84.0,109300.0,1.0,0.0,4.0,5.0,49628.0,12635.0,76.4,0.0,0.0,126.0,294.0,24.0,3.0,4.0,30.0,NaN,1.0,NaN,0.0,4.0,6.0,4.0,4.0,11.0,7.0,7.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5000.0,5000.0,5000.0,36 months,17.97%,180.69,D,D1,NaN,MORTGAGE,62000.0,Not Verified,Mar-19,Issued,n,home_improvement,Home improvement,320xx,FL,19.82,1.0,Nov-87,0.0,4.0,NaN,8.0,0.0,7285.0,77.50%,18.0,w,5000.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Apr-19,Apr-19,0.0,4.0,1.0,Indiv

# Cleaning the target column

In [8]:
train_df = train_df[~train_df["loan_status"].isin(["Fully Paid", "Charged Off", "Issued"])]
train_df["loan_status"] = train_df["loan_status"].replace({"Current": "low-risk"})

status_dict = dict.fromkeys(['In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)', 'Default'], "high-risk")
train_df["loan_status"] = train_df["loan_status"].replace(status_dict)
train_df["loan_status"].unique()

array(['low-risk', 'high-risk', nan], dtype=object)

In [9]:
test_df = test_df[~test_df["loan_status"].isin(["Fully Paid", "Charged Off", "Issued"])]
test_df["loan_status"] = test_df["loan_status"].replace({"Current": "low-risk"})

status_dict = dict.fromkeys(['In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)', 'Default'], "high-risk")
test_df["loan_status"] = test_df["loan_status"].replace(status_dict)
test_df["loan_status"].unique()

array(['low-risk', 'high-risk', nan], dtype=object)

In [10]:
train_df = train_df[~train_df["loan_status"].isna()]
test_df = test_df[~test_df["loan_status"].isna()]

# Cleaning up columns by converting numeric columns that are currently objects

In [11]:
train_df["int_rate"] = train_df["int_rate"].str.replace("%", "").astype(float)/100
test_df["int_rate"] = test_df["int_rate"].str.replace("%", "").astype(float)/100

In [12]:
train_df["revol_util"] = train_df["revol_util"].str.replace("%", "").astype(float)/100
test_df["revol_util"] = test_df["revol_util"].str.replace("%", "").astype(float)/100

In [13]:
train_df["term"] = train_df["term"].str.replace(" months", "").astype(float)
test_df["term"] = test_df["term"].str.replace(" months", "").astype(float)

In [14]:
str_dict = dict(zip(['6 years', '10+ years', '1 year', '5 years', '8 years',
                     '< 1 year', '4 years', '2 years', '3 years', '7 years', '9 years'],
    [6, 10, 1, 5, 8, 0, 4, 2, 3, 7, 9]))

str_dict

{'6 years': 6,
 '10+ years': 10,
 '1 year': 1,
 '5 years': 5,
 '8 years': 8,
 '< 1 year': 0,
 '4 years': 4,
 '2 years': 2,
 '3 years': 3,
 '7 years': 7,
 '9 years': 9}

In [15]:
train_df["emp_length"] = train_df["emp_length"].str.strip().replace(str_dict).astype(float)
test_df["emp_length"] = test_df["emp_length"].str.strip().replace(str_dict).astype(float)

# Drop columns with more than 25% NaN values

In [16]:
# Removing columns with predominantly NaN values
train_df.isna().sum()

id                       391722
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
                          ...  
settlement_status        391716
settlement_date          391716
settlement_amount        391716
settlement_percentage    391716
settlement_term          391716
Length: 140, dtype: int64

In [17]:
del_cols = train_df.count()[train_df.count() < 391722*0.25]

In [18]:
for col in list(del_cols.index):
    train_df.drop(col, axis = 1, inplace = True)

In [19]:
for col in list(del_cols.index):
    test_df.drop(col, axis = 1, inplace = True)

# Creating train and test data frames with dummy columns

In [20]:
train_with_dummies = pd.get_dummies(train_df, dummy_na = True, drop_first = True, sparse = True)
train_with_dummies.head()

loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  installment  \
82     20000.0      20000.0          20000.0  36.0    0.2250       769.00   
93     10500.0      10500.0          10500.0  36.0    0.1719       375.35   
99     25000.0      25000.0          25000.0  36.0    0.2000       929.09   
132    20000.0      20000.0          20000.0  60.0    0.2000       529.88   
133    10000.0      10000.0          10000.0  36.0    0.1640       353.55   

     emp_length  annual_inc    dti  delinq_2yrs  inq_last_6mths  \
82         10.0     75000.0  24.37          0.0             0.0   
93          9.0     66000.0  27.24          0.0             0.0   
99          8.0    105000.0  20.23          0.0             0.0   
132         NaN     56000.0  24.26          0.0             0.0   
133        10.0     92000.0  31.44          0.0             1.0   

     mths_since_last_delinq  open_acc  pub_rec  revol_bal  revol_util  \
82                      NaN       8.0      0.0    44934.0       0.956   
93                     34.0       8.0      0.0     1609.0       0.435   
99                     50.0      17.0      1.0    18368.0       0.364   
132                    66.0       8.0      0.0    13247.0       0.757   
133                     NaN      10.0      1.0    17996.0       0.782   

     total_acc  out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
82        11.0   19431.00       19431.00       719.00           719.00   
93        14.0   10204.87       10204.87       355.29           355.29   
99        34.0   24293.13       24293.13       873.53           873.53   
132       21.0   19636.79       19636.79       485.44           485.44   
133       23.0    9719.34        9719.34       335.33           335.33   

     total_rec_prncp  total_rec_int  total_rec_late_fee  recoveries  \
82            569.00         150.00                 0.0         0.0   
93            295.13          60.16                 0.0         0.0   
99            706.87         166.66                 0.0         0.0   
132           363.21         122.23                 0.0         0.0   
133           280.66          54.67                 0.0         0.0   

     collection_recovery_fee  last_pymnt_amnt  collections_12_mths_ex_med  \
82                       0.0           769.00                         0.0   
93                       0.0           375.35                         0.0   
99                       0.0           929.09                         0.0   
132                      0.0           529.88                         0.0   
133                      0.0           353.55                         0.0   

     policy_code  acc_now_delinq  tot_coll_amt  tot_cur_bal  open_acc_6m  \
82           1.0             0.0           0.0      55014.0          0.0   
93           1.0             0.0         178.0      38199.0          3.0   
99           1.0             0.0         165.0     228119.0          3.0   
132          1.0             0.0           0.0      45684.0          2.0   
133          1.0             0.0           0.0      68784.0          0.0   

     open_act_il  open_il_12m  open_il_24m  mths_since_rcnt_il  total_bal_il  \
82           2.0          0.0          0.0                50.0       10080.0   
93           4.0          2.0          4.0                 3.0       36590.0   
99           3.0          2.0          3.0                 4.0       42273.0   
132          3.0          2.0          3.0                 4.0       32437.0   
133          4.0          1.0          3.0                 9.0       50788.0   

     il_util  open_rv_12m  open_rv_24m  max_bal_bc  all_util  \
82      33.0          1.0          1.0     18068.0      71.0   
93      59.0          1.0          2.0      1553.0      58.0   
99      86.0          4.0          8.0      5197.0      61.0   
132     75.0          2.0          3.0      2810.0      75.0   
133     66.0          0.0          1.0      6360.0      69.0   

     total_rev_hi_lim  inq_fi  total_cu_tl

In [21]:
test_with_dummies = pd.get_dummies(test_df, dummy_na = True, drop_first = True, sparse = True)
test_with_dummies.head()

loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  installment  \
1     5000.0       5000.0           5000.0  36.0    0.1862       182.33   
2    40000.0      40000.0          40000.0  36.0    0.1240      1336.23   
3    32800.0      32800.0          32800.0  36.0    0.1081      1070.89   
4    10000.0      10000.0          10000.0  36.0    0.1612       352.17   
5    17900.0      17900.0          17875.0  36.0    0.2455       707.45   

   emp_length  annual_inc    dti  delinq_2yrs  inq_last_6mths  \
1         NaN     36000.0  26.33          0.0             1.0   
2         5.0    325000.0   7.84          0.0             2.0   
3         6.0     60000.0  20.92          0.0             0.0   
4         2.0     75000.0  15.70          1.0             2.0   
5         4.0    130000.0  32.17          1.0             0.0   

   mths_since_last_delinq  open_acc  pub_rec  revol_bal  revol_util  \
1                     NaN       5.0      0.0    16452.0       0.709   
2                     NaN       8.0      0.0    39290.0       0.423   
3                     NaN       6.0      1.0     2296.0       0.110   
4                     3.0       9.0      0.0    19145.0       0.874   
5                     8.0      22.0      0.0    68977.0       0.313   

   total_acc  out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
1        7.0    3996.20        3996.20      1635.80          1635.80   
2       53.0   31340.28       31340.28     11998.51         11998.51   
3       13.0   27845.98       27845.98      7476.53          7476.53   
4       18.0    7930.09        7930.09      3160.57          3160.57   
5       41.0   14562.98       14542.64      6342.64          6333.78   

   total_rec_prncp  total_rec_int  total_rec_late_fee  recoveries  \
1          1003.80         632.00                 0.0         0.0   
2          8659.72        3338.79                 0.0         0.0   
3          4954.02        2522.51                 0.0         0.0   
4          2069.91        1090.66                 0.0         0.0   
5          3337.02        3005.62                 0.0         0.0   

   collection_recovery_fee  last_pymnt_amnt  collections_12_mths_ex_med  \
1                      0.0              0.0                         0.0   
2                      0.0              0.0                         0.0   
3                      0.0              0.0                         0.0   
4                      0.0              0.0                         0.0   
5                      0.0              0.0                         0.0   

   policy_code  acc_now_delinq  tot_coll_amt  tot_cur_bal  open_acc_6m  \
1          1.0             0.0           0.0      34683.0          0.0   
2          1.0             0.0           0.0      87740.0          0.0   
3          1.0             0.0           0.0      19721.0          0.0   
4          1.0             0.0           0.0     203750.0          0.0   
5          1.0             0.0           0.0     230500.0          0.0   

   open_act_il  open_il_12m  open_il_24m  mths_since_rcnt_il  total_bal_il  \
1          1.0          1.0          1.0                12.0       18231.0   
2          1.0          0.0          1.0                22.0       48450.0   
3          2.0          0.0          2.0                13.0       17425.0   
4          3.0          0.0          0.0                53.0       22833.0   
5          1.0          1.0          2.0                12.0       53006.0   

   il_util  open_rv_12m  open_rv_24m  max_bal_bc  all_util  total_rev_hi_lim  \
1     90.0          0.0          1.0      8141.0      80.0           23200.0   
2      NaN          3.0          4.0     14531.0      42.0           92900.0   
3     55.0          0.0          0.0      2208.0      38.0           20800.0   
4     61.0          1.0          1.0      9558.0      71.0           21900.0   
5     91.0          1.0          2.0     15764.0      37.0          220300.0   

   inq_fi  total_cu_tl  inq_last_12m  ac

In [22]:
train_cols = list(train_with_dummies.columns)
print("train_cols", len(train_cols))

test_cols = list(test_with_dummies.columns)
print("test_cols", len(test_cols))

train_cols 1873
test_cols 1741


In [23]:
train_with_dummies["loan_status"] = train_with_dummies["loan_status_low-risk"]
train_with_dummies.drop("loan_status_low-risk", axis = 1, inplace = True)

test_with_dummies["loan_status"] = test_with_dummies["loan_status_low-risk"]
test_with_dummies.drop("loan_status_low-risk", axis = 1, inplace = True)

# Add missing dummy columns to testing set

In [24]:
for i, col in enumerate(train_cols):
    try:
        test_with_dummies[col]
    except KeyError:
        test_with_dummies[col] = 0
    if i %1000 == 0:
        print(i)
        
#     if col not in test_cols:
#         test_with_dummies[col] = 0

0
1000


# Import additional dependencies for supervised learning

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, confusion_matrix

# Split target from the rest of the data set

In [26]:
y_train = train_with_dummies["loan_status"]
X_train = train_with_dummies.drop("loan_status", axis = 1)
X_train = X_train.fillna(X_train.mean())

y_test = test_with_dummies["loan_status"]
X_test = test_with_dummies.drop("loan_status", axis = 1)[X_train.columns]
X_test = X_test.fillna(X_train.mean())

In [27]:
# Target values are imbalanced
y_train.value_counts()

1    380649
0     11073
Name: loan_status, dtype: int64

# Logistic Regression Model

In [28]:
# Train the Logistic Regression model on the unscaled data and print the model score
clf_lr = LogisticRegression(random_state = 0)
clf_lr.fit(X_train, y_train)

y_pred = clf_lr.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.5001834002965669

# Random Forest Classifier

In [29]:
# Train a Random Forest Classifier model and print the model score
clf_rf = RandomForestClassifier(n_estimators = 10, random_state = 0)
clf_rf.fit(X_train, y_train)

y_pred = clf_rf.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.7720268307811202

# Train models on scaled data

In [30]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [31]:
# numeric_columns = train_df.select_dtypes(include = "number").columns
scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.fit_transform(X_test)

In [32]:
# Train the Logistic Regression model on the scaled data and print the model score
clf_lr = LogisticRegression(random_state = 0)
clf_lr.fit(scaled_train, y_train)

y_pred = clf_lr.predict(scaled_test)

balanced_accuracy_score(y_test, y_pred)

0.5054547970340519

In [33]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf_rf = RandomForestClassifier(n_estimators = 10, random_state = 0)
clf_rf.fit(scaled_train, y_train)

y_pred = clf_rf.predict(scaled_test)

balanced_accuracy_score(y_test, y_pred)

0.7685026717783097